In [1]:
import os
from dotenv import load_dotenv

In [16]:

load_dotenv()

os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    product_name: str = Field(description="Could be any product")
    product_details: str = Field(description="Description about the product")
    tentative_price: float = Field(gt=0.0, description="The tentative price must be in USD")

In [6]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser(pydantic_object=Product)
output_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"product_name": {"description": "Could be any product", "title": "Product Name", "type": "string"}, "product_details": {"description": "Description about the product", "title": "Product Details", "type": "string"}, "tentative_price": {"description": "The tentative price must be in USD", "exclusiveMinimum": 0, "title": "Tentative Price", "type": "number"}}, "required": ["product_name", "product_details", "tentative_price"]}\n```'

In [12]:
from langchain_core.prompts import ChatPromptTemplate

# Escape curly braces by doubling them
format_instructions = output_parser.get_format_instructions().replace("{", "{{").replace("}", "}}")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"You are a helpful product assistant. {format_instructions}"),
        ("user", "{query}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful product assistant. The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}\nthe object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.\n\nHere is the output schema:\n```\n{{"properties": {{"product_name": {{"description": "Could be any product", "title": "Product Name", "type": "string"}}, "product_details": {{"description": "Description about the product", "title": "Product Details", "type": "string"}}, "tentative_price": {{"description": "The t

In [9]:
from langchain_anthropic.chat_models import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-latest")
model

ChatAnthropic(model='claude-3-5-sonnet-latest', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})

In [13]:
chain = prompt | model | output_parser
chain

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful product assistant. The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}\nthe object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.\n\nHere is the output schema:\n```\n{{"properties": {{"product_name": {{"description": "Could be any product", "title": "Product Name", "type": "string"}}, "product_details": {{"description": "Description about the product", "title": "Product Details", "type": "string"}}, "tentative_price": {{"description": "The t

In [14]:
response = chain.invoke(
    {"query": "Can you tell me about the Sony WH-1000XM5 headphones?"}
)

print(response)

{'product_name': 'Sony WH-1000XM5', 'product_details': 'The Sony WH-1000XM5 is a premium wireless noise-canceling headphone featuring industry-leading noise cancellation technology with eight microphones and two processors. It offers up to 30 hours of battery life, multipoint Bluetooth connectivity, and superior sound quality with 30mm drivers. The headphones come with touch controls, speak-to-chat functionality, and adaptive sound control. They feature a new minimalist design with a stepless slider and soft-fit leather for enhanced comfort during long listening sessions.', 'tentative_price': 399.99}


In [15]:
response

{'product_name': 'Sony WH-1000XM5',
 'product_details': 'The Sony WH-1000XM5 is a premium wireless noise-canceling headphone featuring industry-leading noise cancellation technology with eight microphones and two processors. It offers up to 30 hours of battery life, multipoint Bluetooth connectivity, and superior sound quality with 30mm drivers. The headphones come with touch controls, speak-to-chat functionality, and adaptive sound control. They feature a new minimalist design with a stepless slider and soft-fit leather for enhanced comfort during long listening sessions.',
 'tentative_price': 399.99}

In this assignment, we successfully built a simple product assistant using a Language Learning Model (LLM) integrated with Pydantic for structured output. The assistant takes a user query about any product and responds with:

* **Product Name**

* **Product Details**

* **Tentative Price in USD (as an float)**

We utilized LangChain's `ChatPromptTemplate` to guide the model output and `JsonOutputParser` with a `Pydantic` model to ensure the response adhered strictly to the expected structure. This approach guarantees reliable and machine-readable output, making the assistant robust and production-ready.

This solution effectively combines prompt engineering, structured parsing, and modern LLM capabilities to fulfill the task requirements